In [2]:
import torch
import torch.nn as nn
# import torch.nn.functional as F
# from torch.nn import Parameter

In [3]:
class ToyNN(nn.Module):
    def __init__(self, input_dim = 100):
        super().__init__()
        self.l1 = nn.Linear(input_dim, input_dim//2)
        self.l2 = nn.Linear(input_dim//2, input_dim//4)
        self.l3 = nn.Linear(input_dim//4, input_dim//8)
        self.l4 = nn.Linear(input_dim//8, 1)
        self.activ = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        return self.activ(x)

In [7]:
def get_n_params(model):
    """ Returns the number of learnable parameters of a network
    """
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [22]:
toy = ToyNN(8)
print(f"Number of parameters : D : {get_n_params(toy)}")
print(toy)

Number of parameters : D : 51
ToyNN(
  (l1): Linear(in_features=8, out_features=4, bias=True)
  (l2): Linear(in_features=4, out_features=2, bias=True)
  (l3): Linear(in_features=2, out_features=1, bias=True)
  (l4): Linear(in_features=1, out_features=1, bias=True)
  (activ): Sigmoid()
)


In [23]:
for i, layer in enumerate(toy.children()):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i+1} : ")
        print("\t Weight :\n", layer.state_dict()['weight'])
        print("\t Bias :\n", layer.state_dict()['bias'])

Layer 1 : 
	 Weight :
 tensor([[-0.0045, -0.2712, -0.0315, -0.3465,  0.0654,  0.3133, -0.1062,  0.0987],
        [ 0.1268,  0.3232, -0.2300,  0.2387, -0.0988,  0.0447,  0.0027,  0.0917],
        [-0.0631, -0.0034,  0.3162, -0.0822, -0.0323, -0.1681, -0.2588,  0.0360],
        [-0.3348,  0.1780,  0.0400, -0.0727, -0.2602,  0.1808,  0.0514, -0.2854]])
	 Bias :
 tensor([-0.3259, -0.0311, -0.0177, -0.0875])
Layer 2 : 
	 Weight :
 tensor([[ 0.4240, -0.1171, -0.3763,  0.3341],
        [ 0.3654, -0.4166, -0.4447, -0.1376]])
	 Bias :
 tensor([0.3327, 0.0155])
Layer 3 : 
	 Weight :
 tensor([[ 0.3798, -0.0861]])
	 Bias :
 tensor([-0.2560])
Layer 4 : 
	 Weight :
 tensor([[0.2864]])
	 Bias :
 tensor([0.1584])


In [28]:
input_example = torch.Tensor([1,1,1,1,1,1,1,1])

torch.onnx.export(
  toy,
  input_example,
  "model.onnx",
  # only needed if you want to specify an opset version > 1.
  custom_opsets={"custom_domain": 2})

TypeError: linear(): argument 'input' (position 1) must be Tensor, not list